# BackTestのチュートリアル

<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/tutorial/breakout_agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

　
## 1. 実装するBotのロジック

Ukiさんの「オープニングレンジ・ブレイクアウト」をお借りして、バックテストのtutorialを作成しました（公開くさだって感謝）。

Binanceのspot/BTCBUSD用に書いていますが、PublicAPIのみ利用のためアカウントは不要で試すことができます。



<blockquote class="twitter-tweet"><p lang="ja" dir="ltr">お待ちかねのロジックを公開。<br>(元祖)ドテン君はブレイクアウトだが通常のHLチャネルブレイクではない。その手法は「オープニングレンジ・ブレイクアウト」と呼ばれるものである。文章での説明は面倒なのでhohetoとの会議資料をそのまま添付する。ストラテジーに関する質問には応対できません。 <a href="https://t.co/LB6mdxVZZo">pic.twitter.com/LB6mdxVZZo</a></p>&mdash; UKI (@blog_uki) <a href="https://twitter.com/blog_uki/status/981768546429448192?ref_src=twsrc%5Etfw">April 5, 2018</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

![](https://pbs.twimg.com/media/DZ_xJQFUMAEOrpS?format=jpg&name=900x900)

## 2. BreakOutAgentの実装

日本語で読み解くと以下のようになります。

「シグナル点灯した次の足の始値」ではなく、シグナル点灯直後の最良値でエントリーすることに変更した以外はほぼ忠実に書き下しています。

1. `Agent.on_clock`を10分毎に呼び出すように設定する(`Agent.clock_interval`)

2. `Agent.on_clock`内で以下の処理を行う。

    1. 前処理
       
       前回のon_clock中でのオーダーが処理中の場合はなにもしない（リターン）
    2. Long/Short判定
      * 現在時刻から2時間足を6本取得する。`session.ohlcv`（6本目の最後の足は未確定足。10分毎に呼ばれるたびにupdateされる）
      (ohlcv[0]-ohlcv[5]へ値を格納)
      * 0-4本目の足のレンジ幅（高値ー安値）の平均値を計算`(ohlcv['high]-ohlcv['low']).mean() * K`し、Kをかけたものを'range_width'へ保存する。
      * 最新足（未確定）の始値〜高値を計算する('diff_high')
      * 最新足（未確定）の始値〜安値を計算する('diff_low')
      * 'range_width', 'diff_high', 'diff_low'の結果からLong/Short判定
    3. オーダー執行
       1. Long判定のとき
          * すでにLongポジションがある場合は何もしない。
          * Shortポジションがあった場合はドテン（最良値、倍サイズ）
          * 上記に当てはまらない場合は最良値、通常サイズでLong
       2. Short判定のとき
           * すでにShortポジションがある場合は何もしない
           * Shortポジションがあった場合はドテン（最良値、倍サイズ）
           * 上記に当てはまらない場合は最良値、通常サイズでShort

## 2. 依存ライブラリのインストール

### 依存ライブラリ
* numpy
* pandas
* bokeh > version 3以上

それぞれpipでインストールしてください。

Collabの場合、bokehはversion2がはいっていて、またbokeh v2に依存するpanelというライブラリ入っている状態です。panelをアンインストールしてbokehをv3へアップデートします。





In [1]:
# Google Clab用です。必要に応じてコメントアウトしてください
! pip install --upgrade pip
! pip install numpy
! pip install pandas
! pip uninstall -y panel
! pip install --upgrade bokeh > 3

# https://github.com/yasstake/rusty-bot/releases/tag/release-0.2.0
# （環境にあわせて修正お願いします）
# Linux/Google Collab用
!  pip install https://github.com/yasstake/rusty-bot/releases/download/release-0.2.0/rbot-0.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 73.4 MB/s eta 0:00:00


In [4]:
# 念の為パッケージのバージョンを確認 bokehが３　になっているかが重要。
! pip list

Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.9
bokeh                         3.0.3
branca

In [5]:
# 必要ライブラリーのimport
import pandas as pd

# rbotは今回提供するbacktestライブラリ
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackRunner
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE
from rbot.chart import Chart

## 3. break out Agent(bot)の実装

#### 時間の扱い

時刻について、以下の基準で設計しています。
* 時刻（ある一点を指す）: UTCでエポックタイムからのマイクロ秒(μs)。時刻０はfromの時は過去全部、toのときは将来全部（≒現在時刻）
* 期間：ほとんどで秒(s)。ログのダウンロード`Market.download`だけ日(day)
  * 例：ohlcvの足の幅などは秒を利用。

#### 実装すべき必須メソッド（BaseAgentクラスをOverride)

Botは`BaseAgent`を継承し以下の３つを実装する必要があります。
* `__init()__()`    クラスのコンストラクタです。上位クラス`BaseAgent`の初期化`super().__init()__()`をわすれないように気をつけてください。
* `clock_interval()` 次の`on_clock()`が呼び出される周期を秒で返します。
* `on_clock(self, time_us, session)` 周期的に呼び出される関数です。`time_us`が現在時刻で、`session`クラスを使って`ohlcv`をとったり`place_order`したりします。

#### オプション
* `on_tick(self, time_us, session, price, side, size)` 全ログイベントを受け取ります。
* `on_update(self, time, session, result)`オーダーが約定したり失効した場合に呼ばれます。resultの中にオーダ情報が含まれています。



In [6]:
class BreakOutAgent(BaseAgent):
    """
        Agentのクラス名は任意。
        BaseAgentを継承し、clock_interval, on_clockを実装する。
    """

    def __init__(self, param_K=1.6):
            """ super().__init()__ で上位クラスの初期化を必ずすること　"""
            super().__init__()
            self.param_K = param_K  # パラメターKを設定する。

    def clock_interval(self):
        """ on_clockが呼び出される間隔を秒で返す。今回は10分毎にする"""
        return 60 * 10
    
    def on_clock(self, time_us, session):
        """ Botのメインロジック。on_clockで設定した秒数毎に呼ばれる """
        # 前処理/ 前回のon_clock中でのオーダーが処理中の場合はなにもしない（リターン）        
        if session.short_order_len or session.long_order_len:
            return 
        
        ############   メインロジック  ###################### 
        ohlcv_df = session.ohlcv(60*60*2, 6)  # 2時間足(60*60*2sec)を６本取得。 最新は６番目。ただし未確定足
        if len(ohlcv_df.index) < 6:           # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv5 = ohlcv_df[:-2]       # 過去５本足（確定）
        range_width = (ohlcv5['high'] - ohlcv5['low']).mean() * self.param_K  # 価格変動レンジの平均を計算 * K

        # Long/Short判定
        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本(未確定)        
        diff_low   =   (ohlcv_latest['open'][0] - ohlcv_latest['low'][0])
        detect_short  = range_width < diff_low

        diff_high  = - (ohlcv_latest['open'][0] - ohlcv_latest['high'][0])  
        detect_long = range_width  < diff_high
        
        ##########  メインロジック中に利用したindicatorのロギング（あとでグラフ化するため保存）    ##############
        self.log_indicator('diff_low', time_us, diff_low)
        self.log_indicator('diff_high', time_us, diff_high)
        self.log_indicator('range_width', time_us, range_width)

        ##########　執行戦略（順方向のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）#########
        ORDER_SIZE = 0.01     # 標準オーダーサイズ(ドテンの場合はx2)
        ORDER_LIFE = 60*10    # オーダーの有効期間（60x10秒=10分)
        
        if detect_long and (not session.long_position_size): 
            if session.short_position_size: # Shortポジションがあった場合はドテン
                session.place_order( #オーダーを発行する
                            'Buy',                      # 'Buy', 'Sell'を選択
                            session.best_buy_price,     # 最後にbuyがtakeされた価格。sell側にはbest_sell_priceを提供。
                                                        # 任意の価格が設定できるがtakeになる価格でもmakeの処理・手数料で処理している。
                            ORDER_SIZE * 2,             # オーダーサイズ BTCBUSDの場合BTC建で指定。ドテンなので倍サイズでオーダー
                            ORDER_LIFE,                 # オーダーの有効期限（秒）。この秒数をこえるとExpireする。
                            'doten Long'                # あとでログで識別できるように任意の文字列が設定できる。
                )
            else:
                session.place_order('Buy', session.best_buy_price, ORDER_SIZE, ORDER_LIFE, 'Open Long')    

        if detect_short and (not session.short_position_size): # short判定のとき
            if session.long_position_size:  # Longポジションがあった場合はドテン
                session.place_order('Sell', session.best_sell_price, ORDER_SIZE * 2, ORDER_LIFE, 'Doten Short')
            else:
                session.place_order('Sell', session.best_sell_price, ORDER_SIZE, ORDER_LIFE, 'Open Short') 


    # 全Tick受け取りたい時は on_tick を実装する。
    #def on_tick(self, time, session, side, price, size):
    #    pass

    ## 約定イベントを受け取りたい時は on_updateを実装する。
    #def on_update(self, time, session, result):
    #    print(str(result))
    #    pass


## 4. バックテスト用データのロード

現在はBianceのSpot取引のみ対応。

`Market.open`でバイナンス（`BN`)の通貨ペアー(`BTCBUSD`)を開く。
その後、`Market.download`で過去n日分のデータをダウンロードする。

2回目以降はディフォルトではダウンロードしない。
再ダウンロードしたい場合は、`Market.download(5, True)`のように再ダウンロードフラグをTrueにする。

ColabだとダウンロードかDiskに制限があるのか１日あたり１分ぐらいかかることがあります。

In [12]:
BACKTEST_PERIOD = 5             # ダウンロード＆バックテストには時間がかかるのでテスト用に少なめに設定

binance = Market.open('BN', 'BTCBUSD')  # binance marketはあとで利用するので保存しておく

Market.download(BACKTEST_PERIOD)             # BACKTEST_PERIODより最新のログデータを差分ダウンロード
#Market.download(BACKTEST_PERIOD, True)      # 再ダウンロード (１日あたり１０秒＋アルファかかります。)


In [13]:
# Marketオブジェクトを表示するとデータの保持期間、保存場所がわかる
# かなり巨大なデータベースになるので、不要になったら場所を確認して手で削除してください。
# 手元の計測では、70日分で15Gありました（１日あたり200Mb)
binance

start,end
1670889600100000,1671407999900000
"""2022-12-13T00:00:00.100000""","""2022-12-18T23:59:59.900000"""
days=,5


In [14]:
# DBのある場所を表示します。かなり大きなファイルになりますので不要になった場合手動で消してください。
binance.file_name

'/root/.local/share/rbot/DB/BN/SPOT-BTCBUSD.db'

## 5. バックテスト実行

注：いまのところ、Spot取引のBinance(BN)、BTCBUSDのペアーしか対応していません。

In [15]:
 
back_runner = BackRunner(
        'BN',           # Binance は BNと省略します。
        'BTCBUSD',      # 通貨ペアーを選択します。
        False           # 注文時に指定するサイズが通貨ペアーの右側通貨の場合True。BinanceのBTCBUSDはBTCでサイズを指定するのでFalse
)

back_runner.maker_fee_rate = 0.1 * 0.01  # maker_feeを指定（0.1%）. takerは未実装。現在は相手板にぶつける注文をしてもmaker_feeが適用される。

### 5.1 バックテスト本番

In [17]:
agent = BreakOutAgent()         # Agentのインスタンスを作ります（あとで利用するので変数に保存しておきます）。

# 5日前から最新までバックテストする例（最新データは２日ぐらい前のため８日間のバックテスト）。
# Collabの無料版では１日あたり5-6秒かかりました。またM1 MacMiniで１日あたり数秒でした。

back_runner.run(
    agent,                      # backtest するagentインスタンスを指定します。
    rbot.DAYS_BEFORE(BACKTEST_PERIOD),       # 開始時刻を指定します(us)。0だとDBにある最初のデータから処理。DAYS_BEFOREはN日まえのtimestampを返すユーティリティ関数です。
    0                           # 終了時刻を指定します(us). 0だとDBにある最後のデータまで処理。
)     

back_runner                     # back testの結果概要が表示されます

実行結果の分析

実行結果は BackRunner.resultに保存されています。pd.DataFrame形式になっていますのでpandasの機能をつかって分析します。

なお、`size`とはオーダー発行したときの単位での大きさ、`volume`は反対側の通貨での大きさという意味で使い分けています。
またドテンの場合、一つのオーダーがポジションのクローズと新規のポジション作成の２つに分けられ、`sub_id`が一つインクリメントされます。

各カラムの意味は以下のとおり

* index(`create_time`)        オーダーが作られた時刻
* `update_time`	            更新時刻（ステータスが変化したときに更新）
* `1order_id`	                オーダーID（自動付与）
* `sub_id`	                オーダーSubID（ドテンの場合、オーダーCloseとOpenに分割されそれぞれIDが付与される）
* `order_side`	            'Buy'または'Sell'
* `status`	                'Open', 'Close', 'Expire'(期限切れでオーダー無効)のどれかの状態をとる
* `open_price`	            ポジションをオープンしたときの価格
* `open_size`	                                        サイズ
* `open_volume`	                                    ボリューム
* `close_price`	            ポジションをクローズしたときの価格
* `close_size`                                        サイズ
* `close_volume`                                      ボリューム
* `order_price`	            オーダで指定した価格
* `order_size`	                                    サイズ
* `order_volume`	                                    ボリューム
* `profit`	                取引による損益
* `fee`	                    取引手数料
* `total_profit`            手数料込みの損益
* `position_change`	        ポジションの変化
* `message`	                オーダー時に指定したメッセージ文字列
* `sum_profit`	            累積損益
* `position`                  現在のポジション






In [18]:
pd.options.display.max_rows=30

df = back_runner.result
df

,update_time,order_id,sub_id,order_side,status,open_price,open_size,open_volume,close_price,close_size,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-14 13:10:00+00:00,2022-12-14 13:10:00.155000+00:00,0000-0001,0,Buy,Open,17914.61,0.01,179.1461,0.00,0.00,...,17914.61,0.01,179.1461,0.0000,0.179146,-0.179146,0.01,Open Long,-0.179146,0.01
2022-12-14 19:10:00+00:00,2022-12-14 19:20:00.188000+00:00,0000-0002,0,Sell,Expire,0.00,0.00,0.0000,0.00,0.00,...,18223.35,0.02,364.4670,0.0000,0.000000,0.000000,0.00,Doten Short,-0.179146,0.01
2022-12-14 19:40:00+00:00,2022-12-14 19:50:00.235000+00:00,0000-0003,0,Sell,Expire,0.00,0.00,0.0000,0.00,0.00,...,17895.43,0.02,357.9086,0.0000,0.000000,0.000000,0.00,Doten Short,-0.179146,0.01
2022-12-15 13:20:00+00:00,2022-12-15 13:30:00.126000+00:00,0000-0004,0,Sell,Expire,0.00,0.00,0.0000,0.00,0.00,...,17655.44,0.02,353.1088,0.0000,0.000000,0.000000,0.00,Doten Short,-0.179146,0.01
2022-12-15 13:50:00+00:00,2022-12-15 13:52:10.407000+00:00,0000-0005,0,Sell,Close,17914.61,0.01,179.1461,17509.66,0.01,...,17509.66,0.01,175.0966,-4.0495,0.175097,-4.224597,-0.01,Doten Short,-4.403743,0.00
2022-12-15 13:50:00+00:00,2022-12-15 13:52:10.407000+00:00,0000-0005,1,Sell,Open,17509.66,0.01,175.0966,0.00,0.00,...,17509.66,0.01,175.0966,0.0000,0.175097,-0.175097,-0.01,Doten Short,-4.578839,-0.01
2022-12-18 18:50:00+00:00,2022-12-18 18:50:00.716000+00:00,0000-0006,0,Buy,Close,17509.66,0.01,175.0966,16744.66,0.01,...,16744.66,0.01,167.4466,7.6500,0.167447,7.482553,0.01,doten Long,2.903714,0.00
2022-12-18 18:50:00+00:00,2022-12-18 18:50:00.716000+00:00,0000-0006,1,Buy,Open,16744.66,0.01,167.4466,0.00,0.00,...,16744.66,0.01,167.4466,0.0000,0.167447,-0.167447,0.01,doten Long,2.736267,0.01


### いろいろ分析

損益などの単位は、オーダー時に指定したサイズの単位の逆側になります。

例：今回はBTCBUSDの組み合わせに対し、BTCのサイズでオーダ発行しているので、BUSDでの損益。


In [19]:
# 損益結果　 
df['sum_profit'][-1]

2.7362674999999834

In [20]:
# 利益のピーク
df["sum_profit"].max()

2.9037140999999833

In [21]:
# 1回の取引の最大利益
df["total_profit"].max()

7.482553400000006

In [22]:
# １回の取引の最大損失
df["total_profit"].min()

-4.224596600000023

In [23]:
# 手数料なしの場合の損益
df['profit'].sum()

3.6004999999999825

In [24]:
# 手数料合計
df['fee'].sum()

0.8642325

In [25]:
# オーダー回数・Expire数の分析
df.groupby(["order_side", "status"])["status"].count()

order_side  status
Buy         Close     1
            Open      2
Sell        Close     1
            Expire    3
            Open      1
Name: status, dtype: int64

In [26]:
# 勝率分析
win_trade_df  = df[ (df["status"] == "Close") &  (0 < df["total_profit"])]
loss_trade_df = df[ (df["status"] == "Close") &  (df["total_profit"] < 0)]

win_trade = len(win_trade_df)
loss_trade = len(loss_trade_df)

print("買った数:", win_trade)
print("負けた数:", loss_trade)
print("勝率:", win_trade/(loss_trade+win_trade))

買った数: 1
負けた数: 1
勝率: 0.5


In [27]:
# 約定時間分析

df['exec_time'] = df['update_time'] - df.index
df[(df['status'] != 'Expire')]['exec_time'].mean()

Timedelta('0 days 00:00:52.480200')

## 6. グラフで確認

### まずは該当期間のOHLCV（ローソク足）チャートを書く

#### OHLCVデータの取得

まず「３.バックテスト用データのダウンロード」でデータをダウンロードしておきます。

`Market.ohlcv(開始timestamp[us], 終了timestamp[us], 足の幅[sec])`
で取得できる。以下の例ではn日前のタイムスタンプを返す`rbot.DAYS_BEFORE`を利用しています。

毎回Tickデータから任意幅のOHLCVを作っているのでかなり重い処理になってしまっていますが、任意の幅のOHLCVがつくれるのでこれだけ利用しても便利だと思う。

出力はpandas.DataFrameで帰ってきます。

In [28]:
# すでに notebookの最初でbinanceインスタンスは以下のコードで作成済み。ここからohlcvを取り出す。
#binance = Market.open("BN", "BTCBUSD")
#Market.download(BACKTEST_PERIOD)

ohlcv = binance.ohlcv(DAYS_BEFORE(BACKTEST_PERIOD), 0, 60*10)

In [29]:
ohlcv

,open,high,low,close,volume
timestamp,,,,,
2022-12-14 10:00:00+00:00,17803.61,17831.00,17802.80,17822.99,276.15344
2022-12-14 10:10:00+00:00,17823.01,17831.00,17813.71,17829.11,535.76056
2022-12-14 10:20:00+00:00,17828.87,17837.75,17820.00,17832.10,499.30947
2022-12-14 10:30:00+00:00,17832.10,17835.99,17820.76,17824.71,466.98389
2022-12-14 10:40:00+00:00,17824.37,17825.93,17811.82,17817.15,437.77752
...,...,...,...,...,...
2022-12-18 23:10:00+00:00,16798.54,16820.81,16797.00,16799.92,656.92372
2022-12-18 23:20:00+00:00,16800.52,16805.00,16787.63,16796.15,437.86534
2022-12-18 23:30:00+00:00,16795.45,16795.96,16774.67,16789.66,417.77017


#### OHLCVチャートの描画

`Chart`クラスを利用してチャートを描く。
引数は、`Chart(横幅、縦幅、olhcvデータ(DataFrame型))`となっています。

`Chart`インスタンスを作成し、`show()`メソッドを呼び出すことで描画されます。

描画にはbokehライブラリ`https://bokeh.org`を使っています。また`Chart`クラスはPythonの`site-package`にインストールされているはずですので、それをみながらサブクラスを作ると自由に描画がカスタマイズできると思います。（インストール先の場所の例:`lib/python3.10/site-packages/rbot/chart/`)

Colabの場合、ここで、
```
AttributeError: unexpected attribute 'overlay' to CrosshairTool, possible attributes are description, dimensions, js_event_callbacks, js_property_callbacks, line_alpha, line_color, line_width, name, subscribed_events, syncable, tags or toggleable
```
というエラーがでたらbokehのバージョンが2で一旦Pythonにロードされてしまった可能性があります。カーネルを再起動してはじめからやり直してください。

In [30]:
chart = Chart(900, 400, ohlcv)
chart.show()

#### OHLCVにBackTestの結果を追加する。

`Chart.draw_result`でバックテストの結果を追加表示できます。

In [31]:
chart = Chart(900, 400, ohlcv)

chart.draw_result(back_runner.result)

chart.show()

### indicatorの表示

Agentの`on_clock`内で`Agent.log_indicator`を使って保存していたindicatorを`Agent.indicator()`を使って取り出すことができます。`DataFrame`形式になっているので表示します。

In [32]:
diff_low = agent.indicator('diff_low')
diff_high = agent.indicator('diff_high')
range_width = agent.indicator('range_width')
range_width

,value
timestamp,
2022-12-14 10:10:00+00:00,131.797333
2022-12-14 10:20:00+00:00,131.797333
2022-12-14 10:30:00+00:00,131.797333
2022-12-14 10:40:00+00:00,130.298667
2022-12-14 10:50:00+00:00,122.094667
...,...
2022-12-18 23:20:00+00:00,75.813333
2022-12-18 23:30:00+00:00,74.900000
2022-12-18 23:40:00+00:00,74.484000


`Chart.new_figure(新しいパネル名, height=パネルの高さ, title=タイトル)`で新しい描画画面を作ります。

作成したパネル名を指定して、`Chart.line`でデータを表示します。

`Chart.line(描画パネル名, DataFrame, x_key=データフレーム中のx軸名, y_key=データフレーム中のy軸名, color=線の色, legend_label=凡例として表示する名前)`

In [33]:
chart = Chart(900, 400, ohlcv)

chart.draw_result(back_runner.result)

# indicatorの表示
## 新しい描画パネルを作成
chart.new_figure('indicator_panel', height=150, title='indicator')

## indicatorパネルにDataFrameを指定して折線グラフを表示
chart.line('indicator_panel', diff_low, x_key='timestamp', y_key='value', color='#ff0000', legend_label='diff_low')
chart.line('indicator_panel', diff_high, x_key='timestamp', y_key='value', color='#00ff00', legend_label='diff_high')
chart.line('indicator_panel', range_width, x_key='timestamp', y_key='value', color='#0030ff', legend_label='range_width')

chart.show()

## 7. まとめ

自分で書くのは百行ぐらいで簡単にバックテストができることが確認できました。

現在はBiannceのspot/BTCBUSDしか対応していませんが、使ってみてご意見お聞かせください。
Twitter(@yasstake)でもgithubでissueあげてもOK.

人気になったらDiscord立ち上げてみたいなー。



## 8. Next Step

いろいろバックテストしてみよう

#### BreakOutAgentのパラメータ変更
以下のパラメータを変えたらどうなるか確認してみよう
1. パラメータK (現在は1.6きめうち)
2. 指値（現在はbestプライス。
   * buyのときに低い価格で指すと利益は上がるがExpire率が上がる。
   * 高い価格で指すとExpire率が下がるが利益が下がる。（この場合、本来はTakerFeeが取られるが現在はMakerFeeで計算）
3. ロット（現在は0.01 BTC)の変更
4. 2時間足から他の足を試してみる。
5. Clockのタイミングを増やす・へらす（現在は10分）
6. バックテスト期間の延長（現在はBACKTEST_PERIODは5日で設定）

さらに独自のIndicatorをつくり、独自ロジックをつくってバックテストしてみよう（OHLCVをベースに判断するロジックならば問題なく作れるはず）